# PRIMO 2022 ML Homework 4<br>
### Estimating the issue priority in the bug tracker

Задача: предсказать, имеет ли проблема высокий приоритет, учитывая ее описание и метаданные.<br>
Данные: взяты с сайта youtrack.jetbrains.com.<br>
Оценка качества: F1 мера

Метрика оценки качества для соревнования: F1 мера для класса 1.<br>
<br>
**Формат файла с предсказаниями<br>**
Для каждой задачи в наборе test итоговый файл с предсказаниями должен содержать два значения: id (id задачи, взятый из столбца `id` в наборе данных test) и предсказание приоритета задачи (1, если приоритет высокий, или 0, если приоритет низкий).<br>
<br>
Файл должен иметь заголовок и следующий формат:<br>
<br>
id,is_high_priority<br>
0,1<br>
1,1<br>
2,1<br>

In [1]:
import pandas as pd
import numpy as np

import warnings
import json
import operator

from tqdm import tqdm

from sklearn.model_selection import train_test_split


from sklearn.model_selection import ParameterGrid
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE, SelectKBest, f_regression
from sklearn.naive_bayes import MultinomialNB


from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, NMF
from sklearn.feature_selection import SelectKBest, SelectPercentile, chi2
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.metrics import classification_report
#np.set_printoptions(precision=3)
#pd.set_option('precision', 3)



In [2]:
dataFrameMapper_data = pd.read_csv('test.csv', index_col='id')

In [3]:
train_data = pd.read_csv('train.csv', index_col='id')

In [4]:
train_data1 = train_data.sample(frac = 1)

In [5]:
chunks= np.array_split(train_data1, 4)

In [6]:
train_data1 = chunks[0]

In [7]:
#chunks= np.array_split(train_data, 2)

In [8]:
#train_data1 = chunks[0]

In [9]:
#train_data1 = train_data.sample(frac = 1)
#chunks = np.array_split(train_data1, 10)
#train_data1 = chunks[0]

In [10]:
train_data1.summary.fillna('', inplace=True)
train_data1.description.fillna('', inplace=True)


In [11]:
dataFrameMapper_data.summary.fillna('', inplace=True)
dataFrameMapper_data.description.fillna('', inplace=True)

In [12]:
json_fields = pd.json_normalize(train_data1.customFields.map(json.loads).map(lambda x: {field['name']: field['value'] for field in x}))
json_fields.set_index(train_data1.index, inplace=True)
json_fields.fillna('',inplace=True)



In [13]:
json_fields['Subsystem.name'].value_counts()

                        5688
User Interface          1665
Editor. Editing Text    1022
Lang. Flash and Flex     899
Build. Maven             805
                        ... 
Core. Licensing            1
Groovy. Compiler           1
JavaX. CDI                 1
Core. Profiler             1
Clouds. CloudBees          1
Name: Subsystem.name, Length: 165, dtype: int64

In [14]:
json_columns = ['Type.name','State.name','Subsystem.name','Assignee.login']
train_data1 = train_data1.join(json_fields[json_columns], how='outer', lsuffix='_left')


train_data1['reporter_name'] = train_data1.reporter.map(json.loads).map(operator.itemgetter('login'))
train_data1['Year'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').year)
train_data1['Month'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').month)
train_data1['Day'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').day)
train_data1['Hour'] = train_data1.created.apply(lambda c: pd.to_datetime(c, unit='ms').hour)
prep_train_data = train_data1.drop(labels=['reporter', 'created', 'customFields', 'links'], axis=1)

In [15]:
prep_train_data

,summary,description,is_high_priority,Type.name,State.name,Subsystem.name,Assignee.login,reporter_name,Year,Month,Day,Hour
id,,,,,,,,,,,,
25-1920680,Automatic recompile does not work in new version,What steps will reproduce the issue?\n1. Impor...,False,Bug,Incomplete,Build. Built-in (JPS),serge,podalv,2018,3,29,20
25-1018684,Using the enter/return key applis the opposite...,For example: when refactoring a field (that ha...,False,Bug,Duplicate,,,CodeMason,2015,9,20,20
25-1188407,Multiple EntityManagerFactory in the project a...,"If the project uses several EntityManager, con...",False,Usability Problem,Submitted,JavaX. JPA,sergeyv,borz,2016,4,20,19
25-787648,HTML code styles are applied incorrectly,I noticed that after switching to IntelliJ 14 ...,False,Bug,Declined,Editor. Formatter,rvishnyakov,mrooding,2014,11,12,15
25-499309,Wrong XSD is associated,In my classpath there is hibernate-entitymanag...,False,Bug,Fixed,Lang. XML,davdeev,kofeka,2013,1,1,19
...,...,...,...,...,...,...,...,...,...,...,...,...
25-1784064,IDEA should use the ArtifactCollection API for...,DependencyResolverImpl currently uses highly i...,False,Bug,Submitted,Build. Gradle,nskvortsov,st.oehme,2017,11,28,10
25-302638,Introduce Constant Not Working,http://www.screenr.com/p05\n\nI try the keyboa...,False,Bug,Fixed,Lang. Flash and Flex,nicity,John.Lindquist,2011,4,19,16
25-679646,Cannot get file history in SVN 1.7 (for some r...,Our project has sources in several repo's. All...,True,Bug,Fixed,Version Control. Subversion,konstantin.kolosovsky,pieter_degraeuwe,2014,3,20,10


In [16]:
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import re

In [17]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\WildGoose\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WildGoose\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\WildGoose\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

*сначала происходит лемматизация слов, и потом они отфильтровываются по условиям, так работать не должно*

In [18]:
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
       return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if re.fullmatch(r'[A-Za-z]{4,}', t) and t not in STOP_WORDS]

In [19]:
X_train_text = prep_train_data.summary.astype(str) + " " + prep_train_data.description.astype(str)


In [20]:
X_train_text

id
25-1920680    Automatic recompile does not work in new versi...
25-1018684    Using the enter/return key applis the opposite...
25-1188407    Multiple EntityManagerFactory in the project a...
25-787648     HTML code styles are applied incorrectly I not...
25-499309     Wrong XSD is associated In my classpath there ...
                                    ...                        
25-1784064    IDEA should use the ArtifactCollection API for...
25-302638     Introduce Constant Not Working http://www.scre...
25-679646     Cannot get file history in SVN 1.7 (for some r...
25-936465     Unicode characters in Editor Editor cant displ...
25-634680     Minimize Horizontal scrolling in project view ...
Length: 26342, dtype: object

In [21]:
tfidf_vec_summary = TfidfVectorizer(min_df=.01, max_df=0.75,
                                    tokenizer=LemmaTokenizer(),
                             )
tfidf_vec_summary.fit(X_train_text)
train_text_prep = pd.DataFrame(tfidf_vec_summary.transform(X_train_text).todense(), index=X_train_text.index,
                               columns=tfidf_vec_summary.get_feature_names_out())
train_text_prep



c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,ability,able,access,action,active,actual,actually,added,adding,additional,...,warning,webstorm,window,word,work,workaround,worked,working,write,wrong
id,,,,,,,,,,,,,,,,,,,,,
25-1920680,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.071580,0.0,0.0,0.000000,0.0,0.000000
25-1018684,0.0,0.0,0.0,0.130999,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.094848,0.0,0.0,0.000000,0.0,0.000000
25-1188407,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
25-787648,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
25-499309,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.531724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-1784064,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.084652,0.0,0.0,0.000000,0.0,0.124597
25-302638,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.340875,0.0,0.000000
25-679646,0.0,0.0,0.0,0.156271,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.113146,0.0,0.0,0.000000,0.0,0.000000


In [22]:
Y = prep_train_data['is_high_priority']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(train_text_prep, Y,
                                                    stratify=Y,
                                                    random_state=3,
                                                    test_size=0.3,
                                                    shuffle=True)

In [24]:
selector = SelectKBest(f_regression, k=2)

selector.fit(X_train, y_train)

print(selector.scores_)

selector.transform(X_train)

[1.99721649e+00 6.75566453e-01 2.06174758e-03 4.38207099e-01
 5.23637656e+00 1.06240636e+00 9.97643806e-01 4.69422613e-02
 3.45864441e-01 1.63734716e+00 7.21728714e+00 1.89072863e+00
 7.51318414e+01 4.47037954e+00 8.88114320e-01 2.42860406e-02
 8.15487035e-02 3.22236361e-01 5.50744098e+00 9.12146776e-01
 2.78143543e-01 2.48759222e+00 8.72735517e-02 7.72545048e-02
 1.27431501e-01 2.96856788e+00 2.91367245e+00 7.00801216e-01
 1.74874338e+00 6.21771362e-01 6.04911641e-01 3.26972530e-05
 1.42961727e-02 2.09828085e-02 8.88765425e-03 1.20545007e+00
 1.66123319e+00 3.87221452e-01 2.95979391e+00 1.20077819e+00
 5.02417824e+00 2.20737135e-01 1.00227764e+00 1.18708835e+00
 1.41982858e+00 2.79046278e+01 4.32749277e+00 8.44926141e-02
 1.85686160e+00 1.49448299e+00 4.14426473e+00 3.18251362e+00
 1.22235802e+01 2.22377713e+01 2.21034513e+00 2.27832739e-01
 2.24338383e-01 4.43654900e+00 9.05180185e-01 3.84710419e+00
 9.02193485e-01 1.30196667e+01 7.75868110e-02 3.05915823e-01
 1.43713576e+00 2.394508

array([[0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       ...,
       [0.11907283, 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]])

In [25]:
prep_train_data = prep_train_data.drop(['summary','description'], axis=1)

In [26]:
prep_train_data = prep_train_data.join(train_text_prep)

In [27]:
prep_train_data

,is_high_priority,Type.name,State.name,Subsystem.name,Assignee.login,reporter_name,Year,Month,Day,Hour,...,warning,webstorm,window,word,work,workaround,worked,working,write,wrong
id,,,,,,,,,,,,,,,,,,,,,
25-1920680,False,Bug,Incomplete,Build. Built-in (JPS),serge,podalv,2018,3,29,20,...,0.0,0.0,0.0,0.0,0.071580,0.0,0.0,0.000000,0.0,0.000000
25-1018684,False,Bug,Duplicate,,,CodeMason,2015,9,20,20,...,0.0,0.0,0.0,0.0,0.094848,0.0,0.0,0.000000,0.0,0.000000
25-1188407,False,Usability Problem,Submitted,JavaX. JPA,sergeyv,borz,2016,4,20,19,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
25-787648,False,Bug,Declined,Editor. Formatter,rvishnyakov,mrooding,2014,11,12,15,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
25-499309,False,Bug,Fixed,Lang. XML,davdeev,kofeka,2013,1,1,19,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.531724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-1784064,False,Bug,Submitted,Build. Gradle,nskvortsov,st.oehme,2017,11,28,10,...,0.0,0.0,0.0,0.0,0.084652,0.0,0.0,0.000000,0.0,0.124597
25-302638,False,Bug,Fixed,Lang. Flash and Flex,nicity,John.Lindquist,2011,4,19,16,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.340875,0.0,0.000000
25-679646,True,Bug,Fixed,Version Control. Subversion,konstantin.kolosovsky,pieter_degraeuwe,2014,3,20,10,...,0.0,0.0,0.0,0.0,0.113146,0.0,0.0,0.000000,0.0,0.000000


In [28]:
#pd.options.display.max_rows = 200



tfidf_vec_desc = TfidfVectorizer(min_df=.01, max_df=0.5,
                                   # token_pattern=r'[A-Za-z]{2,}',
                                    tokenizer=LemmaTokenizer(),
                                    stop_words=STOP_WORDS)
tfidf_vec_desc.fit(X_train_description)
X_train_description_sample = pd.DataFrame(tfidf_vec_desc.transform(X_train_description).todense(),
                                          columns=tfidf_vec_desc.get_feature_names_out())
X_train_description_sample

\# todo сократить размерность. Лемматизация, регулярные выражения поумнее, пороги для векторизации, объединить слова описания и заголовка?
отбор признаков по модели

In [29]:
X, Y = prep_train_data.drop(['is_high_priority'], axis=1), prep_train_data['is_high_priority']


In [30]:
X_ohe = pd.get_dummies(X,prefix=['Type'], columns = ['Type.name'])

In [31]:
X_ohe = pd.get_dummies(X,prefix=['Subsystem'], columns = ['Subsystem.name'], drop_first=True)

In [32]:
X_ohe = pd.get_dummies(X,prefix=['State'], columns = ['State.name'], drop_first=True)

In [33]:
X_ohe = pd.get_dummies(X,prefix=['Assignee'], columns = ['Assignee.login'], drop_first=True)

In [34]:
X_ohe = pd.get_dummies(X,prefix=['Reporter'], columns = ['reporter_name'], drop_first=True)

In [35]:
X_ohe=X_ohe.drop(['Type.name','State.name','Subsystem.name','Assignee.login'], axis=1)

In [36]:
#dataFrameMapper_data = preprocessor.fit_transform(X)


In [37]:
#dataFrameMapper_data.to_csv('dataMapperData.csv')

In [38]:
#Y = Y.astype(np.int8)
X_ohe

,Year,Month,Day,Hour,ability,able,access,action,active,actual,...,Reporter_Павел_Швец,Reporter_Павлин_Краснопёров,Reporter_Петр_Королев,Reporter_Сергей.Камалтынов,Reporter_Станислав.Алёкминский,Reporter_Стас.Бицько,Reporter_Тимур_Шакуров,Reporter_Фагим.Садыков,Reporter_дэн.человек,Reporter_震_田
id,,,,,,,,,,,,,,,,,,,,,
25-1920680,2018,3,29,20,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
25-1018684,2015,9,20,20,0.0,0.0,0.0,0.130999,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
25-1188407,2016,4,20,19,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
25-787648,2014,11,12,15,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
25-499309,2013,1,1,19,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25-1784064,2017,11,28,10,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
25-302638,2011,4,19,16,0.0,0.0,0.0,0.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
25-679646,2014,3,20,10,0.0,0.0,0.0,0.156271,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_ohe, Y,
                                                    stratify=Y,
                                                    random_state=3,
                                                    test_size=0.25,
                                                    shuffle=True)

In [40]:
from sklearn.feature_selection import SelectFromModel

In [41]:
selector = SelectKBest(chi2)
selector.fit_transform(X_train,y_train)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [44]:
param_grid = dict(threshold=[.05,.0023, 0.01, 0.07, 0.08], selector = [LogisticRegression(),KNeighborsClassifier(n_neighbors=2),ExtraTreesClassifier()] )

resultsProbaSelectModel = []
for params in tqdm(ParameterGrid(param_grid)):

    clf = Pipeline([
        ('feature_selection', SelectFromModel(LinearSVC())),
        ('classification', RandomForestClassifier())
    ])
    clf.fit(X_train, y_train)

    resultsProbaSelectModel.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=clf.predict_proba(X_train)[:,1] > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=clf.predict_proba(X_test)[:,1] > params['threshold']),

    ))

  0%|          | 0/15 [00:00<?, ?it/s]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
  7%|▋         | 1/15 [01:02<14:33, 62.38s/it]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
 13%|█▎        | 2/15 [02:01<13:09, 60.70s/it]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
 20%|██        | 3/15 [03:02<12:05, 60.44s/it]c:\users\wildgoose\.virtualenvs\data science\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
 27%|██▋       | 4/15 [04:00<10:55, 59.64s/it]c:\users\w

In [45]:
pd.DataFrame(resultsProbaSelectModel).sort_values('test_f1').style.bar(vmin=0, vmax=1)

,parameters,train_f1,test_f1
1,"{'selector': LogisticRegression(), 'threshold': 0.0023}",0.168175,0.097253
6,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.0023}",0.164791,0.098731
11,"{'selector': ExtraTreesClassifier(), 'threshold': 0.0023}",0.169545,0.101506
12,"{'selector': ExtraTreesClassifier(), 'threshold': 0.01}",0.289933,0.106065
2,"{'selector': LogisticRegression(), 'threshold': 0.01}",0.292485,0.109175
7,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.01}",0.292733,0.109321
13,"{'selector': ExtraTreesClassifier(), 'threshold': 0.07}",0.833575,0.122105
4,"{'selector': LogisticRegression(), 'threshold': 0.08}",0.857143,0.122402
5,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.05}",0.712284,0.123743
9,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.08}",0.855022,0.124294


In [43]:
pd.DataFrame(resultsProba).sort_values('test_f1').style.bar(vmin=0, vmax=1)

,parameters,train_f1,test_f1
12,"{'selector': ExtraTreesClassifier(), 'threshold': 0.01}",0.085220,0.081572
2,"{'selector': LogisticRegression(), 'threshold': 0.01}",0.085178,0.081856
6,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.0023}",0.084972,0.081909
7,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.01}",0.085174,0.082127
1,"{'selector': LogisticRegression(), 'threshold': 0.0023}",0.084918,0.082228
11,"{'selector': ExtraTreesClassifier(), 'threshold': 0.0023}",0.084906,0.082438
13,"{'selector': ExtraTreesClassifier(), 'threshold': 0.07}",0.190345,0.103734
8,"{'selector': KNeighborsClassifier(n_neighbors=2), 'threshold': 0.07}",0.189300,0.107216
10,"{'selector': ExtraTreesClassifier(), 'threshold': 0.05}",0.187883,0.109980
14,"{'selector': ExtraTreesClassifier(), 'threshold': 0.08}",0.191136,0.111801


In [ ]:
pred = clf.predict_proba(X_test)[:,1]

In [ ]:
threshold = 0.07
f1_score(y_true=y_test, y_pred= pred > threshold)

In [ ]:
# https://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB, MultinomialNB, ComplementNB

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = MinMaxScaler()

X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

X_train_scaled

# todo надо бужет скэйлить!
from sklearn.ensemble import RandomForestRegressor
clf = SGDClassifier()

clf.fit(X_train_scaled,y_train)

In [ ]:
SVC_model = SVC()
# В KNN-модели нужно указать параметр n_neighbors
# Это число точек, на которое будет смотреть
# классификатор, чтобы определить, к какому классу принадлежит новая точка
KNN_model = KNeighborsClassifier(n_neighbors=5)
RFC_model = RandomForestClassifier()

SVC_model.fit(X_train, y_train)
KNN_model.fit(X_train, y_train)
RFC_model.fit(X_train, y_train)

SVC_prediction = SVC_model.predict(X_test)
KNN_prediction = KNN_model.predict(X_test)
RFC_model = RandomForestClassifier.predict_proba(X_test)[:,1]

In [ ]:
logReg = KNeighborsClassifier(n_neighbors=2)
logReg.fit(X_train_scaled, y_train)

In [ ]:
predictions = logReg.predict(X_test_scaled)

In [ ]:
proba = logReg.predict_proba(X_test_scaled)[:, 1]

In [ ]:
threshold = 0.03
f1_score(y_true=y_test, y_pred=proba > threshold)

In [ ]:
f1_score(y_true=y_test, y_pred=KNN_prediction)

In [ ]:
param_grid = dict(threshold=[.1, .2, .3, .4, .5, .01,.005, .05, .001,.0023, 0.003])

resultsProba = []
for params in tqdm(ParameterGrid(param_grid)):

  #  prediction = logReg.predict_proba(X_test_scaled)[:,1]
    resultsProba.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=logReg.predict_proba(X_train_scaled)[:,1] > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=logReg.predict_proba(X_test_scaled)[:,1] > params['threshold']),

    ))


In [ ]:
param_grid = dict(threshold=[.1, .2, .3, .4, .5, .01,.005, .05, .001,.0023, 0.003])

resultsProba = []
for params in tqdm(ParameterGrid(param_grid)):

  #  prediction = logReg.predict_proba(X_test_scaled)[:,1]
    resultsProba.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=logReg.predict_proba(X_train_scaled)[:,1] > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=logReg.predict_proba(X_test_scaled)[:,1] > params['threshold']),

    ))


In [ ]:
pd.DataFrame(resultsProba).sort_values('test_f1').style.bar(vmin=0, vmax=1)



prediction = clf.predict_proba(X_test)[:,1]
#print (f'Test f1 = {f1_score(y_true=y_test, y_pred=nb_pipeline.predict(X_test))}')

prediction

score = clf.score(X_test, y_test)
score

In [ ]:
X_test

In [ ]:

minMaxTest_Y = prep_train_data.is_high_priority

In [ ]:
test_minMaxdf_X = prep_train_data.summary.astype(str) + " " + prep_train_data.description.astype(str)

In [ ]:
X_train_minmax, X_test_minmax, y_train_minmax, y_test_minmax = train_test_split(test_minMaxdf_X, minMaxTest_Y,
                                                    stratify=Y,
                                                    random_state=3,
                                                    test_size=0.25,
                                                    shuffle=True)

In [ ]:
X_train_minmax

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
param_grid = dict(min_df=[0.005, 0.01,.05, 0.06], max_df=[.5,.6,.7,.8], classifier=[ComplementNB(),LogisticRegression()], threshold=[.15,.05, 0.1])

results = []

for params in tqdm(ParameterGrid(param_grid)):
#     print(params)
    pipe = Pipeline(steps = [ ('tfid', TfidfVectorizer(min_df= params['min_df'], max_df= params['max_df'],
                                    tokenizer=LemmaTokenizer(),
                             )),
                             ('to_dense', DenseTransformer()),
                             ('clf',  params['classifier'])
                            ]
                       )

    #cv_train_preds = cross_val_predict(pipe, X_text_train, y_train, cv = 5, n_jobs = -1)

    pipe.fit(X_train_minmax, y_train_minmax)


    probabilitiesText1 = pipe.predict_proba(X_train_minmax)[:, 1]
    probabilitiesText2 = pipe.predict_proba(X_test_minmax)[:, 1]

    results.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=probabilitiesText1 > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=probabilitiesText2 > params['threshold'])
    ))


In [ ]:
pd.DataFrame(results).sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [ ]:
pd.DataFrame(results).drop(columns='estimator').sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [ ]:


#param_grid = dict(classifier=[ComplementNB(),LogisticRegression(),LinearRegression(), RandomForestRegressor()])




nb_pipeline = Pipeline(steps = [('scaler', MinMaxScaler()),
                                ('clf', ComplementNB())
                               ]
                       )

nb_pipeline.fit(X_train, y_train)

print (f'Train f1 = {f1_score(y_true=y_train, y_pred=nb_pipeline.predict(X_train))}')
print (f'Test f1 = {f1_score(y_true=y_test, y_pred=nb_pipeline.predict(X_test))}')

In [ ]:
param_grid = dict(threshold=[.1, .2, .3, .4, .5, .01,.005, .05])

results = []
for params in tqdm(ParameterGrid(param_grid)):
    probabilities1 = nb_pipeline.predict_proba(X_train)[:, 1]
    probabilities2 = nb_pipeline.predict_proba(X_test)[:, 1]
    results.append(dict(
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=probabilities1 > params['threshold']),
                        test_f1 = f1_score(y_true=y_test, y_pred=probabilities2 > params['threshold'])
    ))



In [ ]:
pd.DataFrame(results).sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [ ]:


from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
param_grid = dict(min_df=[0.01, .05], max_df=[.7, .8], scaler=[MinMaxScaler()])

results = []

for params in tqdm(ParameterGrid(param_grid)):
#     print(params)
    pipe = Pipeline(steps = [
        ('scaler', params['scaler']),
                             ('clf', ComplementNB())
                            ]
                       )

    #cv_train_preds = cross_val_predict(pipe, X_text_train, y_train, cv = 5, n_jobs = -1)

    pipe.fit(X_train, y_train)
    train_preds = pipe.predict(X_test)

    results.append(dict(
                        estimator=pipe,
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=pipe.predict(X_train)),
                        test_f1 = f1_score(y_true=y_test, y_pred=pipe.predict(X_test))
    ))


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.linear_model import LinearRegression, LogisticRegression, BayesianRidge, PassiveAggressiveRegressor, SGDClassifier
param_grid = dict(min_df=[0.01, .05], max_df=[.7, .8])

results = []

for params in tqdm(ParameterGrid(param_grid)):
#     print(params)
    pipe = Pipeline(steps = [('tfidf', TfidfVectorizer(min_df=params['min_df'], max_df=params['max_df'],
                                                       token_pattern=r'[A-Za-z]{2,}',
                                                       stop_words=STOP_WORDS)),
                             ('to_dense', DenseTransformer()),
                             ('clf', ComplementNB())
                            ]
                       )

    #cv_train_preds = cross_val_predict(pipe, X_text_train, y_train, cv = 5, n_jobs = -1)

    pipe.fit(X_train, y_train)
    train_preds = pipe.predict(X_train)

    results.append(dict(
                        estimator=pipe,
                        parameters=params,
                        train_f1 = f1_score(y_true=y_train, y_pred=pipe.predict(X_train)),
                        test_f1 = f1_score(y_true=y_test, y_pred=pipe.predict(X_test))
    ))

In [ ]:
pd.DataFrame(results).drop(columns='estimator').sort_values('test_f1').style.bar(vmin=0, vmax=1)

In [ ]:
train_data1

In [ ]:
probabilities = pipe.predict_proba(X)[:, 1]

In [ ]:
probabilities

In [ ]:
threshold = 0.6

In [ ]:
f1_score(y_true=y, y_pred=probabilities > threshold)